In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# ✅ Load API key from .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# ✅ Step 1: Load FAISS index
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(
    "faiss_index_notice_privacy",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True  # required for index.pkl
)

# ✅ Step 2: Set up retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",  # or "mmr"
    search_kwargs={"k": 5}      # top 5 chunks
)

# ✅ Step 3: Initialize LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=openai_api_key
)

# ✅ Step 4: Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,  # optional: useful for debugging
    chain_type="stuff"
)

# ✅ Step 5: Interactive chatbot loop
print("📄 PDF ChatBot is ready. Ask any question about your documents.")
while True:
    user_query = input("\n🧠 Your question (or type 'exit'): ")
    if user_query.lower() in ("exit", "quit"):
        print("👋 Goodbye!")
        break

    try:
        result = qa_chain(user_query)
        print(f"\n✅ Answer:\n{result['result']}")
        
        # Optional: show sources
        print("\n📚 Sources:")
        for doc in result["source_documents"]:
            print(f"- Page {doc.metadata['page']} from {doc.metadata['source']}")

    except Exception as e:
        print(f"\n❌ Error: {e}")


/Users/sanket/Documents/Python Projects/New Project LLM MultiDB/langchain_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sanket/Documents/Python Projects/New Project LLM MultiDB/langchain_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/sanket/Documents/Python Projects/New Project LLM MultiDB/langchain_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use

📄 PDF ChatBot is ready. Ask any question about your documents.


/Users/sanket/Documents/Python Projects/New Project LLM MultiDB/langchain_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



✅ Answer:
If you forget your My UNC Chart username or password, you can click on the "Forgot Username?" or "Forgot Password?" link below the login fields for assistance. You will be prompted to provide your first and last name, date of birth, and zip code to verify your identity and recover your username or password. If your email address is on file, your username will be sent to that email. If you can't remember the required information or don't have a valid email address on file, you will need to contact the UNC Health Outpatient Access Center at (888) 996-2767 for assistance.

📚 Sources:
- Page 4 from User Guide.pdf
- Page 42 from User Guide.pdf
- Page 43 from User Guide.pdf
- Page 3 from User Guide.pdf
- Page 4 from User Guide.pdf
👋 Goodbye!
